# Sparse Extended Infromation Filter SLAM

In [ ]:
import numpy as np
from scipy.stats import norm

import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
%matplotlib notebook

In [ ]:
class SEIF_SLAM:
    show_animation = True
    
    # simulation parameters
    time_dt = 0.1     # time step [s]
    time_sim = 50.0   # total simulation time [s]
    ndim_state = 3    # state dimension [x, y, yaw]
    ndim_landmark = 2 # Landmark state dimension [x, y]
    sensor_max_range = 10.0 # maximum observation range of the simulated sensor
    m_dist_threshold = 3  # Threshold of Mahalanobis distance for data association.

    # EKF state covariance
    R_t = np.diag([0.5, 0.5, np.deg2rad(30.0)])**2

    #  Simulation parameter
    Qsim = np.diag([0.2, np.deg2rad(1.0)])**2   # Covariance of process noise
    Rsim = np.diag([1.0, np.deg2rad(10.0)])**2  # Covariance of measurement noise

    # landmark positions [x, y]
    landmarks = np.array(
        [[10.0, -2.0],
         [13.0, 10.0],
         [ 3.0, 8.0],
         [-5.0, 12.0],
         [ 8.0,  4.0],
         [ 4.0,  4.0],
         [ 2.0,  4.0],
         [-5.0, 10.0],
         [10.0, 14.0]]
    )

    def __init__(self):
        if self.show_animation:
            marker_config = {
                'markersize': 16
            }
            line_config = {
                'linewidth': 4
            }
            self.fig, self.ax = plt.subplots(figsize=[10,10])
            # plot ground truth landmarks
            self.line_lm, = plt.plot(self.landmarks[:, 0], self.landmarks[:, 1], "*k", **marker_config)
            # plot estimated of agent's location
            self.line_x_hat, = plt.plot(0, 0, ".r", **marker_config)
            # plot detection range
            self.circle_x = plt.Circle([0,0], self.sensor_max_range, fill=False)
            self.ax.add_artist(self.circle_x)
            # plot estimated landmarks
            self.line_lm_hat = []
            for i in range(self.landmarks.shape[0]):
                line, = plt.plot(0, 0, "xg", **marker_config)
                self.line_lm_hat.append(line)
            # plot histories
            self.line_hx, = plt.plot(0, 0, "-b", **line_config)
            self.line_hx_dr, = plt.plot(0, 0, "-k", **line_config)
            self.line_hx_hat, = plt.plot(0, 0, "-r", **line_config)
            
            plt.axis("equal")
            plt.grid(True)
            plt.xlim([-20, 20])
            plt.ylim([-20, 20])
            plt.legend(
                [
                    self.line_hx, self.line_hx_dr, self.line_hx_hat,
                    self.line_x_hat, 
                    self.line_lm, self.line_lm_hat[0]
                ],
                [
                    'Agent path (truth)', 
                    'Agent path (DR)',
                    'Agent path (estimated)',
                    'Agent location (estimated)',
                    'Landmarks (truth)', 'Landmarks (estimated)'
                ],
                loc='best',
                fontsize=10
            )
            self.fig.canvas.draw()
    
    def run(self):
        # state Vector [x y yaw]'
        x_hat = np.matrix(np.zeros((self.ndim_state, 1)))
        x = np.matrix(np.zeros((self.ndim_state, 1)))
        # state covariance estimate (initialize to identity)
        P_hat = np.eye(self.ndim_state)
        # Dead reckoning
        x_dr = np.matrix(np.zeros((self.ndim_state, 1)))
        # history
        h_x = x
        h_x_hat = x_hat
        h_x_dr = x
        
        if self.show_animation:
            self.line_lm.set_xdata(self.landmarks[:, 0])
        time = 0.0
        while self.time_sim >= time:
            time += self.time_dt
            # get new control
            u = self.get_u()
            # take new observation
            x, z, x_dr, ud = self.observation(x, x_dr, u, self.landmarks)
            # update state and covariance
            x_hat, P_hat = self.ekf_slam(x_hat, P_hat, ud, z)
            # get agent state
            x_state = x_hat[0:self.ndim_state]

            # store data history
            h_x     = np.hstack((h_x, x))
            h_x_hat = np.hstack((h_x_hat, x_state))
            h_x_dr  = np.hstack((h_x_dr, x_dr))
            
            if self.show_animation:
                # estimated of agent's location
                self.line_x_hat.set_xdata(x_hat[0])
                self.line_x_hat.set_ydata(x_hat[1])
                self.circle_x.set_center(x_hat[:2])
                # estimated landmarks
                for i in range(self.count_landmarks(x_hat)):
                    self.line_lm_hat[i].set_xdata(x_hat[self.ndim_state + i*2])
                    self.line_lm_hat[i].set_ydata(x_hat[self.ndim_state + i*2 + 1])
                self.line_hx.set_xdata(np.array(h_x[0, :]).flatten())
                self.line_hx.set_ydata(np.array(h_x[1, :]).flatten())
                self.line_hx_dr.set_xdata(np.array(h_x_dr[0, :]).flatten())
                self.line_hx_dr.set_ydata(np.array(h_x_dr[1, :]).flatten())
                self.line_hx_hat.set_xdata(np.array(h_x_hat[0, :]).flatten())
                self.line_hx_hat.set_ydata(np.array(h_x_hat[1, :]).flatten())
                plt.pause(0.01)
                self.fig.canvas.draw()
        
    def get_u(self):
        v = 1.0        # [m/s]
        yawrate = 0.15  # [rad/s]
        u = np.matrix([v, yawrate]).T
        return u

    def motion_model(self, x, u):
        # https://en.wikipedia.org/wiki/State-space_representation
        # state matrix 3x3
        A = np.matrix([[1.0, 0, 0],
                       [0, 1.0, 0],
                       [0, 0, 1.0]])
        # input matrix 3x2
        theta = x[2, 0]
        B = np.matrix([[self.time_dt * np.cos(theta), 0],
                       [self.time_dt * np.sin(theta), 0],
                       [0.0, self.time_dt]])
        # update state vector
        x = A * x + B * u
        return x
    
    def count_landmarks(self, x):
        # count number of landmarks in x
        n = (len(x) - self.ndim_state) // self.ndim_landmark
        return n

    def get_landmark_pos(self, x, z):
        '''
        Compute landmark position from range-bearing measurement
        and current agent's state estimate
        '''
        zp = np.zeros((2, 1))
        zp[0, 0] = x[0, 0] + z[0, 0] * np.cos(x[2, 0] + z[0, 1])
        zp[1, 0] = x[1, 0] + z[0, 0] * np.sin(x[2, 0] + z[0, 1])
        return zp

    def get_landmark_pos_from_state(self, x, idx):
        # get the idx-th landmark position from state matrix
        lm = x[(self.ndim_state + self.ndim_landmark * idx) : (self.ndim_state + self.ndim_landmark * (idx + 1)), :]
        return lm

    def offset_angle(self, angle):
        # make sure angle is from 0 to 2*pi
        return (angle + np.pi) % (2 * np.pi) - np.pi

    def search_landmark_id(self, xAug, PAug, zi):
        '''
        Find and return the landmark that minimizes the Mahalanobis distance from the agent
        '''
        nLM = self.count_landmarks(xAug)
        min_dist = self.m_dist_threshold
        min_idx = nLM
        for i in range(nLM):
            lm = self.get_landmark_pos_from_state(xAug, i)
            y, S, H = self.calc_innovation(i, lm, xAug, PAug, zi)
            mdist = y.T * np.linalg.inv(S) * y
            if mdist < min_dist:
                min_dist = mdist
                min_idx = i
            
        return min_idx

    def jacobian_motion(self, x, u):
        '''
        Jacobian of motion function g
        '''
        # mapping matrix Fx helps map actual state vector (dim_state)
        # into combined state vector (dim_state + dim_landmark*cnt_landmark)
        Fx = np.hstack(
            (
                np.eye(self.ndim_state),
                np.zeros((self.ndim_state, self.ndim_landmark * self.count_landmarks(x)))
            )
        )
        theta = x[2, 0]
        Delta = np.matrix([[0.0, 0.0, -self.time_dt * u[0] * np.sin(theta)],
                        [0.0, 0.0,   self.time_dt * u[0] * np.cos(theta)],
                        [0.0, 0.0, 0.0]])
        return Delta
    
    def jacobian_H(self, q, sq, delta, x, i):
        '''
        Jacobian of measurement function h
        
        INPUT:
            q : squared delta_k
            sq: square root of q
            delta: location error between landmark i and agent
            x: current state
            i: index of current landmark
            
        OUTPUT:
            H: Jacobian of measurement function h
        '''
        dx = delta[0, 0]
        dy = delta[1, 0]
        G = np.matrix(
            [
                [-sq * dx, -sq * dy, 0, sq * dx, sq * dy],
                [dy, -dx, -1.0, -dy, dx]
            ]
        )
        G = G / q
        # mapping matrix Fx helps map actual state vector (dim_state)
        # into combined state vector (dim_state + dim_landmark*cnt_landmark)
        num_landmarks = self.count_landmarks(x)
        Fx = np.hstack(
            (
                np.eye(self.ndim_state),
                np.zeros((self.ndim_state, self.ndim_landmark * num_landmarks))
            )
        ) # upper half of F_xk
        Fk = np.hstack(
            (
                np.zeros((self.ndim_landmark, self.ndim_state)), 
                np.zeros((self.ndim_landmark, self.ndim_landmark * (i - 1))),
                np.eye(self.ndim_landmark), 
                np.zeros((self.ndim_landmark, self.ndim_landmark * num_landmarks - self.ndim_landmark * i))
            )
        ) # lower half of F_xk
        F_xk = np.vstack((Fx, Fk))
        H = G * F_xk
        return H
   
    def observation(self, x, x_hat, u, landmarks):
        # update new true state
        x = self.motion_model(x, u)

        # measurement noise
        z = np.matrix(np.zeros((0, self.ndim_state)))
        # loop through all landmarks and append to measurement matrix if valid
        num_landmarks = len(landmarks[:, 0])
        for i in range(num_landmarks):
            dx = landmarks[i, 0] - x[0, 0]  # x distance from i-th landmark to the position
            dy = landmarks[i, 1] - x[1, 0]  # y distance from i-th landmark to the position
            d = np.sqrt(dx**2 + dy**2)      # true l-2 distance between landmark and current true position
            angle = self.offset_angle(np.arctan2(dy, dx)) # true angle measurement
            if d <= self.sensor_max_range:
                # if the landmark is in the detactable range of the sensor
                d_hat = d + np.random.randn() * self.Qsim[0, 0]          # measured distance plus process noise
                angle_hat = angle + np.random.randn() * self.Qsim[1, 1]  # measured angle plus process noise
                # put together the measurement matrix for the i-th landmark
                zi = np.matrix([d_hat, angle_hat, i]) # measurement of i-th landmark
                z = np.vstack((z, zi))                # all measurements

        # add control noise to all input u dimensions
        ndim_u = len(u[:, 0])
        u_dims = []
        for i in range(ndim_u):
            u_dims.append(u[i, 0] + np.random.randn() * self.Rsim[i, i])    
        u_hat = np.matrix(u_dims).T
        # predict new state with predicted x and u
        # this is the dead reckoning state estimation x_dr
        x_hat = self.motion_model(x_hat, u_hat)
        return x, z, x_hat, u_hat
    
    def calc_innovation(self, landmark_idx, landmark, x_hat, P_hat, z):
        # location delta
        delta = landmark - x_hat[0:2] 
        # predicted distance measurement
        q = (delta.T * delta)[0, 0]   
        sq = np.sqrt(q)
        # predicted angle measurement
        z_angle = np.arctan2(delta[1], delta[0]) - x_hat[2] 
        # put together, z_hat = predicted measurement vector
        z_hat = [sq, self.offset_angle(z_angle)]
        # innovation
        y = (z - z_hat).T
        y[1] = self.offset_angle(y[1])
        # Jacobian of measurement function
        H = self.jacobian_H(q, sq, delta, x_hat, landmark_idx + 1)
        # compute S, P_hat is the estimated covariance of measurement
        # and R_t is the pre-defined covariance of state 
        # (we are only using) the upper-left 2-by-2 
        S = H * P_hat * H.T + self.R_t[0:2, 0:2]
        
        return y, S, H
    
    def ekf_slam(self, x_hat, P_hat, u, z):
        # Predict
        nS = self.ndim_state
        x_hat[0:nS] = self.motion_model(x_hat[0:nS], u)
        G, Fx = self.jacobian_motion(x_hat[0:nS], u)
        P_hat[0:nS, 0:nS] = G.T * P_hat[0:nS, 0:nS] * G + Fx.T * self.R_t * Fx
        P_init = np.eye(2) # initialize covariance matrix to identify
        
        # Update
        ndim_z = len(z[:, 0])
        for iz in range(ndim_z):  # for each observation
            lm_id = self.search_landmark_id(x_hat, P_hat, z[iz, 0:2])
            nLM = self.count_landmarks(x_hat)
            if lm_id == nLM:
                print(f'Found new landmark (total {nLM+1})')
                # Extend state and covariance matrix
                x_tmp = np.vstack((x_hat, self.get_landmark_pos(x_hat, z[iz, :])))
                P_hat = np.vstack(
                    (
                        np.hstack((P_hat, np.zeros((len(x_hat), self.ndim_landmark)))),
                        np.hstack((np.zeros((self.ndim_landmark, len(x_hat))), P_init))
                    )
                )
                x_hat = x_tmp
            
            lm = self.get_landmark_pos_from_state(x_hat, lm_id)
            y, S, H = self.calc_innovation(lm_id, lm, x_hat, P_hat, z[iz, 0:2])
            # Kalman gain
            K = P_hat * H.T * np.linalg.inv(S)
            x_hat = x_hat + K * y
            P_hat = (np.eye(len(x_hat)) - K * H) * P_hat

        x_hat[2] = self.offset_angle(x_hat[2])

        return x_hat, P_hat


ekf = EKF_SLAM()
ekf.run()